In [1]:
%load_ext autoreload
%autoreload 2

## Generate Final Extended Metaphor - GPT 3.5 Turbo

In [2]:
import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', 0)
import json
import re

import pathlib
import sys

from openai import OpenAI
import os

# Add src module to path before import.
sys.path.insert(0, str(pathlib.Path('../../src')))

from file_IO_handler import get_plaintext_file_contents, save_json, load_json
from fill_string_template import get_filled_strings_from_dataframe, FilledString

## Get Access to OpenAI API

In [3]:
OPENAI_API_KEY = get_plaintext_file_contents(pathlib.Path("../../OPENAI_API_KEY.env"))
# print(OPENAI_API_KEY)

MODEL_NAME = "gpt-3.5-turbo"


client = OpenAI(api_key=OPENAI_API_KEY)


def Completion(prompt, is_json=False, is_short_ans=False):
    if is_json:
        return client.chat.completions.create(
            model="gpt-3.5-turbo",
            response_format={ "type": "json_object" },
            messages=[{"role": "system", "content": "You are a helpful assistant designed to output JSON."}, {"role": "user", "content": prompt}]
        )
    elif is_short_ans:
        return client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "Output one word or phrase"}, {"role": "user", "content": prompt}]
        )
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

## Prepare Inputs

In [4]:
df = pd.read_csv(pathlib.Path("../generations/script_run_GPT35Turbo/prompt_7_output.csv"), index_col=0)
df.columns

Index(['level_of_difficulty', 'tensor_name', 'source_domain', 'subtensor_name',
       'subtensor_definition', 'subtensor_purpose', 'subtensor_mechanism',
       'text', 'text_annotated', 'text_redacted', 'schema', 'target_domain',
       'subvehicle_name', 'subtensor_name_as_json_key',
       'subvehicle_name_as_json_key', 'extended_metaphor', 'final_output'],
      dtype='object')

In [5]:
df[["tensor_name", "source_domain", "subtensor_name", "target_domain", "subvehicle_name"]]

,tensor_name,source_domain,subtensor_name,target_domain,subvehicle_name
0,solar system,Astronomy,Planets,Kpop,Choreography
1,solar system,Astronomy,Planets,baseball,Fielding positions and coordination.
2,solar system,Astronomy,Planets,Iron Man,Repulsor Technology
3,solar system,Astronomy,Planets,money,Cryptocurrency
4,solar system,Astronomy,Moons,Kpop,Concept of fan interactions within the Kpop industry
...,...,...,...,...,...
359,superstring theory,Physics,String compactification,money,Blockchain
360,superstring theory,Physics,D-branes,Kpop,Concept Music Videos
361,superstring theory,Physics,D-branes,baseball,Pitcher's mound
362,superstring theory,Physics,D-branes,Iron Man,Nanotechnology


In [6]:
def is_like(row):
    if row['subtensor_name'].endswith("s"):
        return row['subtensor_name'] + " are like " + row['subvehicle_name'] + "."
    return row['subtensor_name'] + " is like " + row['subvehicle_name'] + "."

In [7]:
df['is_like'] = df.apply(is_like, axis=1)
df['list_of_is_like'] = df[['tensor_name','target_domain', 'is_like']].groupby(['tensor_name','target_domain'])['is_like'].transform(lambda x: ' '.join(x))
df['list_of_subtensors'] = df[['tensor_name','target_domain', 'subtensor_name']].groupby(['tensor_name','target_domain'])['subtensor_name'].transform(lambda x: ', '.join(x))
df['list_of_subvehicles'] = df[['tensor_name','target_domain', 'subvehicle_name']].groupby(['tensor_name','target_domain'])['subvehicle_name'].transform(lambda x: ', '.join(x))
df['list_of_extended_metaphor'] = df[['tensor_name','target_domain', 'extended_metaphor']].groupby(['tensor_name','target_domain'])['extended_metaphor'].transform(lambda x: ','.join(x))
df_slice = df[[
    'level_of_difficulty',
    'tensor_name',
    'source_domain',
    'target_domain', 
    'list_of_is_like', 
    'list_of_subtensors', 
    'list_of_subvehicles', 
    'list_of_extended_metaphor',
]].drop_duplicates()
len(df_slice)

84

In [8]:
df_slice.head(1)

level_of_difficulty   tensor_name source_domain target_domain  \
0  Elementary          solar system  Astronomy     Kpop           

                                                                                                                                                                                                   list_of_is_like  \
0  Planets are like Choreography. Moons are like Concept of fan interactions within the Kpop industry. Asteroids are like Choreography. Comets are like Choreography aspects. Dwarf Planets are like Choreography.   

                                 list_of_subtensors  \
0  Planets, Moons, Asteroids, Comets, Dwarf Planets   

                                                                                                    list_of_subvehicles  \
0  Choreography, Concept of fan interactions within the Kpop industry, Choreography, Choreography aspects, Choreography   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Save Locations

In [9]:
# Save every call
prompt_use_relationships_output_save_to_prefix = pathlib.Path("../generations/script_run_GPT35Turbo/prompt_use_relationships_output")
prompt_cover_subtensors_output_save_to_prefix = pathlib.Path("../generations/script_run_GPT35Turbo/prompt_cover_subtensors_output")

# Consolidated to csv
prompt_use_relationships_extended_metaphors_output_save_to = pathlib.Path("../generations/script_run_GPT35Turbo/prompt_use_relationships_extended_metaphors_output.csv")
prompt_cover_subtensors_extended_metaphors_output_save_to = pathlib.Path("../generations/script_run_GPT35Turbo/prompt_cover_subtensors_extended_metaphors_output.csv")

## Prepare Prompts

In [10]:
prompt_use_relationships_style_template = get_plaintext_file_contents(pathlib.Path("../prompt_template/prompt_use_relationships_write_extended_metaphor.txt"))
prompt_cover_subtensors_style_template = get_plaintext_file_contents(pathlib.Path("../prompt_template/prompt_cover_subtensors_write_extended_metaphor.txt"))

In [11]:
print(prompt_use_relationships_style_template)

Write a concise extended metaphor comparing ${tensor_name} to ${target_domain}.

Use the following relationships: ${list_of_is_like}

Use the following information:
```
${list_of_extended_metaphor}
```


In [12]:
def run_prompt_use_relationships_extended_metaphors(filled_strings):
    pre_df = {
        "level_of_difficulty": [],
        "tensor_name": [],
        "source_domain": [],
        "target_domain": [],
        "list_of_is_like": [],
        "list_of_subtensors": [], 
        "list_of_subvehicles": [],
        "list_of_extended_metaphor": [],
        "use_relationships_extended_metaphor": [],
    }

    # for idx in range(len(filled_strings)):
    idx = 0
    save_path = pathlib.Path(f"{prompt_use_relationships_output_save_to_prefix}_idx_{idx}.txt")

    level_of_difficulty = filled_strings[idx].values["level_of_difficulty"]
    tensor_name = filled_strings[idx].values["tensor_name"]
    source_domain = filled_strings[idx].values["source_domain"]
    target_domain = filled_strings[idx].values["target_domain"]
    list_of_is_like = filled_strings[idx].values["list_of_is_like"]
    list_of_subtensors = filled_strings[idx].values["list_of_subtensors"]
    list_of_subvehicles = filled_strings[idx].values["list_of_subvehicles"]
    list_of_extended_metaphor = filled_strings[idx].values["list_of_extended_metaphor"]

    pre_df["level_of_difficulty"].append(level_of_difficulty)
    pre_df["tensor_name"].append(tensor_name)
    pre_df["source_domain"].append(source_domain)
    pre_df["target_domain"].append(target_domain)
    pre_df["list_of_is_like"].append(list_of_is_like)
    pre_df["list_of_subtensors"].append(list_of_subtensors)
    pre_df["list_of_subvehicles"].append(list_of_subvehicles)
    pre_df["list_of_extended_metaphor"].append(list_of_extended_metaphor)
    
    if save_path.exists():
        print(f"Loading from {save_path}")
        with open(save_path, 'r') as file:
            out_content = file.read()
    else:
        # call model
        out = Completion(filled_strings[idx].filled, is_json=False)
        out_content = out.choices[0].message.content
        
        # save output
        with open(save_path, 'w+') as file:
            file.write(out_content)
        print(f"Called model and saved to {save_path}")

    pre_df["use_relationships_extended_metaphor"].append(out_content)

    df_out = pd.DataFrame(pre_df)
    df_out.to_csv(prompt_use_relationships_extended_metaphors_output_save_to)
    
    return df_out

In [13]:
print(prompt_cover_subtensors_style_template)

Write a concise extended metaphor comparing ${tensor_name} to ${target_domain}.

Cover the following aspects: ${list_of_subtensors}


In [14]:
def run_prompt_cover_subtensors_extended_metaphors(filled_strings):
    pre_df = {
        "level_of_difficulty": [],
        "tensor_name": [],
        "source_domain": [],
        "target_domain": [],
        "list_of_is_like": [],
        "list_of_subtensors": [], 
        "list_of_subvehicles": [],
        "list_of_extended_metaphor": [],
        "cover_subtensors_extended_metaphor": [],
    }

    # for idx in range(len(filled_strings)):
    idx = 0
    save_path = pathlib.Path(f"{prompt_cover_subtensors_output_save_to_prefix}_idx_{idx}.txt")

    level_of_difficulty = filled_strings[idx].values["level_of_difficulty"]
    tensor_name = filled_strings[idx].values["tensor_name"]
    source_domain = filled_strings[idx].values["source_domain"]
    target_domain = filled_strings[idx].values["target_domain"]
    list_of_is_like = filled_strings[idx].values["list_of_is_like"]
    list_of_subtensors = filled_strings[idx].values["list_of_subtensors"]
    list_of_subvehicles = filled_strings[idx].values["list_of_subvehicles"]
    list_of_extended_metaphor = filled_strings[idx].values["list_of_extended_metaphor"]

    pre_df["level_of_difficulty"].append(level_of_difficulty)
    pre_df["tensor_name"].append(tensor_name)
    pre_df["source_domain"].append(source_domain)
    pre_df["target_domain"].append(target_domain)
    pre_df["list_of_is_like"].append(list_of_is_like)
    pre_df["list_of_subtensors"].append(list_of_subtensors)
    pre_df["list_of_subvehicles"].append(list_of_subvehicles)
    pre_df["list_of_extended_metaphor"].append(list_of_extended_metaphor)
    
    if save_path.exists():
        print(f"Loading from {save_path}")
        with open(save_path, 'r') as file:
            out_content = file.read()
    else:
        # call model
        out = Completion(filled_strings[idx].filled, is_json=False)
        out_content = out.choices[0].message.content
        
        # save output
        with open(save_path, 'w+') as file:
            file.write(out_content)
        print(f"Called model and saved to {save_path}")

    pre_df["cover_subtensors_extended_metaphor"].append(out_content)

    df_out = pd.DataFrame(pre_df)
    df_out.to_csv(prompt_cover_subtensors_extended_metaphors_output_save_to)
    
    return df_out

## Run - Use Relationships

In [15]:
filled_strings = get_filled_strings_from_dataframe(prompt_use_relationships_style_template, df_slice)
print(filled_strings[0].filled)

Write a concise extended metaphor comparing solar system to Kpop.

Use the following relationships: Planets are like Choreography. Moons are like Concept of fan interactions within the Kpop industry. Asteroids are like Choreography. Comets are like Choreography aspects. Dwarf Planets are like Choreography.

Use the following information:
```
[
  {
    "metaphor": "both Planets in the solar system and Choreography in Kpop have Gravitational Equilibrium",
    "planets": "Gravitational Equilibrium in the solar system contributes to overall stability by maintaining precise geometric positions and movements of celestial bodies.",
    "choreography": "Similarly, Choreography in Kpop maintains equilibrium by ensuring precise movements and positions of dancers on stage, contributing to the overall stability and aesthetics of the performance."
  },
  {
    "metaphor": "both Planets in the solar system and Choreography in Kpop have Elliptical Paths",
    "planets": "Elliptical Paths dictate the 

In [16]:
df_use_relationships = run_prompt_use_relationships_extended_metaphors(filled_strings)

Called model and saved to ../generations/script_run_GPT35Turbo/prompt_use_relationships_output_idx_0.txt


In [23]:
print(df_use_relationships[["use_relationships_extended_metaphor"]].iloc[0, 0])

The solar system can be likened to the world of Kpop, where each element plays a vital role in creating a harmonious and visually appealing experience for fans. Just as planets in the solar system rely on choreography for Gravitational Equilibrium, ensuring precise movements and positions of dancers on stage, Kpop choreography establishes equilibrium by maintaining stability and aesthetics in performances. Moons, like the Concept of fan interactions within the Kpop industry, provide stabilization and maintain equilibrium, forming a supportive community that keeps the industry thriving. Asteroids, similar to Choreography in Kpop, utilize Gravitational Forces and Orbital Mechanics to create balance and visually appealing performances through strategic positioning and movements. Comets and Choreography aspects share Orbital Mechanics, using movement forces and Material Science to craft visually appealing routines that seamlessly flow in performances. Lastly, Dwarf Planets, akin to Choreog

## Run - Cover Subtensors

In [18]:
filled_strings = get_filled_strings_from_dataframe(prompt_cover_subtensors_style_template, df_slice)
print(filled_strings[0].filled)

Write a concise extended metaphor comparing solar system to Kpop.

Cover the following aspects: Planets, Moons, Asteroids, Comets, Dwarf Planets


In [19]:
df_cover_subtensors = run_prompt_cover_subtensors_extended_metaphors(filled_strings)

Called model and saved to ../generations/script_run_GPT35Turbo/prompt_cover_subtensors_output_idx_0.txt


In [24]:
print(df_cover_subtensors[["cover_subtensors_extended_metaphor"]].iloc[0, 0])

In the vast galaxy of entertainment, Kpop can be likened to a solar system, with various components that make it a unique and vibrant entity.

The planets represent the biggest Kpop acts, shining brightly and drawing in fans from all across the world. These planets, like BTS and BLACKPINK, are at the center of attention, with their music and performances captivating audiences everywhere.

The moons orbiting these planets are the loyal fans, dedicated and supportive of their favorite artists. Just as moons reflect the light of the sun, these fans reflect the talent and passion of the Kpop idols they admire.

Asteroids in the Kpop solar system are the rising stars, emerging with potential and talent that could one day shine just as bright as the planets. Groups like TXT and Aespa are on the cusp of greatness, poised to make a big impact in the industry.

Comets are the unexpected surprises in Kpop, artists or groups that burst onto the scene with a unique sound or concept. They may not a